In [25]:
import numpy as np
import pandas as pd

team_stats = {}
for year in range(2011, 2017):
    off = pd.read_csv('csv/team_stats/off/'+str(year)+'.csv').sort_values(by=['Team'])
    off.columns = ['Off'+col if col!='Team' else col for col in off.columns]
    defense = pd.read_csv('csv/team_stats/def/'+str(year)+'.csv').sort_values(['Team'])
    defense.columns = ['Def'+col for col in defense.columns]
    off_col = [off.columns.values[1]]+list(off.columns.values[4:])
    def_col = list(defense.columns.values)[4:]
    team_stats[year] = pd.concat([off[off_col], defense[def_col]], axis=1)
team_stat_names = list(team_stats[2011].columns.values)[1:]
opp_stat_names = ['Opp'+name for name in team_stat_names]
column_names = team_stat_names + opp_stat_names
column_names.append('YrsSince2011')
column_names.append('Home')

In [26]:
def make_x(train_or_test):
    data = pd.read_csv('games_'+train_or_test+'.csv', delimiter=',', header=None)
    features = pd.DataFrame(index=range(data.shape[0]*2), columns = column_names)
    for data_ind, game in data.iterrows():
        year = game[4]
        away_team = game[0]
        home_team = game[2]
        for team_ind,team in team_stats[year-1].iterrows():
            if team['Team']==away_team:
                away_team_stats=team[team_stat_names]
            if team['Team']==home_team:
                home_team_stats=team[team_stat_names]
        away_row = np.concatenate((away_team_stats.append(home_team_stats).as_matrix(),[year-2011],[0]))
        home_row = np.concatenate((home_team_stats.append(away_team_stats).as_matrix(),[year-2011],[1]))
        features.loc[2*data_ind] = away_row
        features.loc[2*data_ind+1] = home_row
    features.to_csv('csv/x_'+train_or_test+'.csv',index=None)
make_x('train')
make_x('test')